In [1]:
import pandas as pd
import pickle
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from statistics import mean
from feature_functions import *
from natasha_func import get_doc

In [2]:
dataset = pickle.load(open("dataset.pkl", "rb"))

In [3]:
def get_feature(dataset, func):
    feature = []
    for book in dataset['books']:
        dok = get_doc(book)
        f = func(dok)
        feature.append(f)
    return feature

In [4]:
def dataset_target(dataset):
    authors = list(set(dataset['authors']))
    dict_authors = dict()
    reverse_dict_authors = dict()
    for index in range(len(authors)):
        dict_authors[authors[index]] = index
        reverse_dict_authors[index] = authors[index]
    target = []
    for author in dataset["authors"]:
        target += [dict_authors[author]]
    return target

In [5]:
target = dataset_target(dataset)
dataset_for_ml = {"targets": target}
df = pd.DataFrame(dataset_for_ml)

In [6]:
functions = [frequency_of_i, frequency_of_comma, frequency_of_dot, frequency_of_exclamation_mark,frequency_of_question_mark, frequency_of_brackets, frequency_of_quotation_marks, frequency_of_numbers, number_of_unique_words, number_of_words_that_occur_once, frequency_of_longest_word, average_lenght_of_sentence_by_letters, frequency_of_words_in_initial_form, correlation_of_vowels_consonants, most_common_first_letter_in_words, most_common_first_letter_in_sents, frequency_of_a, frequency_of_o, frequency_of_е, frequency_of_n, frequency_of_t, average_length_of_help_words, len_of_most_common_word, average_lenght_of_nouns, average_lenght_of_adjectives, average_lenght_of_adverbs, average_lenght_of_verbs, correlation_of_short_long_words, correlation_of_long_medium_words, semicolon_freq, count_upper_words, freq_of_freq_word, freq_word_from_adjective, freq_word_from_noun, freq_word_from_verbs, freq_of_space, sentences_avg_len_symbols, capitalized_words_count_without_start_of_sentences, avg_syllable_per_noun, avg_syllable_per_verb, avg_syllable_per_adjective, avg_syllable_per_adverb, count_words_infinitive]
for func in functions:
    feature = get_feature(dataset, func)
    df[func.__name__] = feature


AttributeError: module 'inspect' has no attribute 'getargspec'

In [ ]:
df.to_csv('out.csv', index=False)

In [ ]:
def validation(df, model):
    accuracy = []
    for shift in range(0, 10, 2):
        authors_seen = []
        test_ixs = []
        for ixs in range(len(df)):
            aut = df['targets'][ixs]
            if authors_seen.count(aut) < 2:
                authors_seen += [aut]
                test_ixs += [ixs + shift]
        train_ixs = list(set(range(len(df))) - set(test_ixs))
        y_test = df.iloc[test_ixs]['targets']
        x_test = df.iloc[test_ixs].drop(columns=["targets"])
        y_train = df.iloc[train_ixs]['targets']
        x_train = df.iloc[train_ixs].drop(columns=["targets"])

        rfc = model
        rfc.fit(x_train, y_train)
        accuracy.append(accuracy_score(y_test, rfc.predict(x_test)))
    return accuracy

print( mean(validation(df, ExtraTreesClassifier())))

In [ ]:
from catboost import CatBoostClassifier

validation(df, CatBoostClassifier())